In [66]:
%load_ext autoreload
%autoreload 2
import datetime
import os
import pandas as pd
print(datetime.datetime.now())

from pygentoolbox.Tools import read_interleaved_fasta_as_noninterleaved
from pygentoolbox.Tools import read_interleaved_fasta_as_noninterleaved_human_proteins
from pygentoolbox.Tools import reverse_complement_to_dna
from pygentoolbox.Tools import translate_AA_from_dna_human
from pygentoolbox.Tools import translate_AA_from_dna_human_terminate_at_stop
#dir(pygentoolbox.Tools)
%matplotlib inline
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2019-09-06 11:30:24.026132


In [70]:
humanannotation = 'D:\\LinuxShare\\Humans\\Genome\\Annotations\\ref_GRCh38.p12_top_level.gff3'
from pygentoolbox.TrimAnnotationByFeature import trim_to_mRNA_and_add_protein_id
trim_to_mRNA_and_add_protein_id(humanannotation)  # Keep only mRNA features, but add the protein ID from the CDS features
## i manually deleted the entries that were not assigned to chromosomes

################################
annotationfile = 'D:\\LinuxShare\\Humans\\Genome\\Annotations\\ref_GRCh38.p12_top_level.mRNA.wProtID.ChromOnly.gff3'
annotation = []
print('Reading in annotation: %s' % annotationfile)
with open(annotationfile, 'r') as FILE:
    annotation = [line.strip().split('\t') for line in FILE if line[0] != '#']  # skip header
print('Finished reading in annotation')

scaffolds = [s[0] for s in annotation]  # collect ordered scaffold names

################################
proteinfile = annotationfile = 'D:\\LinuxShare\\Humans\\Genome\\Seqs\\protein.fa'
print('Reading in "normal" proteins: %s' % proteinfile)
namesproteins, dproteins = read_interleaved_fasta_as_noninterleaved_human_proteins(proteinfile)
print('Finished reading in proteins')

################################
genomefile = 'D:\\LinuxShare\\Humans\\Genome\\Seqs\\GRCh38_top_level.fa'
print('Reading in genome: %s' % genomefile)
names, dgenome = read_interleaved_fasta_as_noninterleaved(genomefile)
print('Finished reading in genome')

print(datetime.datetime.now())

Output file: D:\LinuxShare\Humans\Genome\Annotations\ref_GRCh38.p12_top_level.mRNA.wProtID.gff3
Reading in annotation: D:\LinuxShare\Humans\Genome\Annotations\ref_GRCh38.p12_top_level.mRNA.wProtID.ChromOnly.gff3
Finished reading in annotation
Reading in "normal" proteins: D:\LinuxShare\Humans\Genome\Seqs\protein.fa
Counting total lines
Number of lines in genome file: 1251806
Reading in interleaved fasta as noninterleaved fasta dictionary
Current line: 1000000
Finished reading in proteins
Reading in genome: D:\LinuxShare\Humans\Genome\Seqs\GRCh38_top_level.fa
Counting total lines
Number of lines in genome file: 45847621
Reading in interleaved fasta as noninterleaved fasta dictionary
Current line: 5000000
Current line: 10000000
Current line: 15000000
Current line: 20000000
Current line: 25000000
Current line: 30000000
Current line: 35000000
Current line: 40000000
Current line: 45000000
Finished reading in genome
2019-09-06 11:34:48.312323


In [71]:
# for forward (+)
# gff3 entry
# NC_000001.11	BestRefSeq	mRNA	69091	70008	.	+	.	ID=rna9;Parent=gene8;Dbxref=GeneID:79501,Genbank:NM_001005484.1,HGNC:HGNC:14825;Name=NM_001005484.1;gbkey=mRNA;gene=OR4F5;product=olfactory receptor family 4 subfamily F member 5;transcript_id=NM_001005484.1
# subtract one from the start position if it is forward (+) to get start codon
# to get stop codon use ending coordinate and subtract 3 for start
# start should match atggtgactg
# end should match agttttag

print('forward +')
print('start')
print(dgenome['NC_000001.11'][69090:69093])
#print(dgenome['NC_000001.11'][70000:70008])
print('end')
print(dgenome['NC_000001.11'][70005:70008])
print('+1 frameshift')
print(dgenome['NC_000001.11'][70006:70009])
peplength = 30
start = 69091
end = 70008
# forward +1 frameshift
print(dgenome['NC_000001.11'][end-2:end+peplength-2])  # end-2:end+1+peplength-3
print('Translate')
print('#####  Positive Control Forward Translation: should match AA sequence  #####')
print('MVTEFIFLGL')
print(translate_AA_from_dna_human(dgenome['NC_000001.11'][start-1:start+peplength-1]))
print('#####  Positive Control Forward Translation  #####')

print('#####  +1 frameshift translation  #####')
PlusOneAAseq = translate_AA_from_dna_human(dgenome['NC_000001.11'][end-2:end+peplength-2])
print(PlusOneAAseq)
PlusOneAAseq = translate_AA_from_dna_human_terminate_at_stop(dgenome['NC_000001.11'][end-2:end+peplength-2])
print(PlusOneAAseq)

# for reverse (-)
# gff3 entry
# NC_000001.11	BestRefSeq	mRNA	450740	451678	.	-	.	ID=rna41;Parent=gene23;Dbxref=GeneID:729759,Genbank:NM_001005221.2,HGNC:HGNC:31275;Name=NM_001005221.2;gbkey=mRNA;gene=OR4F29;product=olfactory receptor family 4 subfamily F member 29;transcript_id=NM_001005221.2
# subtract one from the start position if it is forward (+)
# to get stop codon use ending coordinate and subtract 3 for start
# start should match atggatggag 
# end should match atctcataa

print('reverse -')
print('end')
print(dgenome['NC_000001.11'][450739:450742])  # 'end of protein'
print('start')
print(dgenome['NC_000001.11'][451675:451678])  # 'start of protein'
# need to reverse complement to get 5' -> 3' order desired
print('+1 frameshift')
print(dgenome['NC_000001.11'][450738:450741])  # 'end of protein'
end = 450740
start = 451678

print('Translate')
print('#####  Positive Control Reverse Translation: should match AA sequence  #####')
print('MDGENHSVVS')
revcomp, rev = reverse_complement_to_dna(dgenome['NC_000001.11'][start-peplength:start])
print(translate_AA_from_dna_human(revcomp))
print('#####  Positive Control Reverse Translation  #####')

# reverse +1 frameshift
print('#####  +1 frameshift translation  #####')
print(dgenome['NC_000001.11'][end-peplength+1:end+1])  # long form # [end-2-peplength+3:end+1
print('Reverse complement, the \'reverse -\' sequence')
revcomp, rev = reverse_complement_to_dna(dgenome['NC_000001.11'][end-peplength+1:end+1])  # long form # [end-2-peplength+3:end+1]
print(revcomp)
PlusOneAAseq = translate_AA_from_dna_human(revcomp)
print(PlusOneAAseq)
PlusOneAAseq = translate_AA_from_dna_human_terminate_at_stop(revcomp)
print(PlusOneAAseq)


forward +
start
ATG
end
TAG
+1 frameshift
AGA
AGATCTTATATAACTGTGAGATTAATCTCA
Translate
#####  Positive Control Forward Translation: should match AA sequence  #####
MVTEFIFLGL
MVTEFIFLGL
#####  Positive Control Forward Translation  #####
#####  +1 frameshift translation  #####
RSYITVRLIS
RSYITVRLIS
reverse -
end
TTA
start
CAT
+1 frameshift
TTT
Translate
#####  Positive Control Reverse Translation: should match AA sequence  #####
MDGENHSVVS
MDGENHSVVS
#####  Positive Control Reverse Translation  #####
#####  +1 frameshift translation  #####
GTTTAATGAGAAGGGCTTATTATATCATTT
Reverse complement, the 'reverse -' sequence
AAATGATATAATAAGCCCTTCTCATTAAAC
K*YNKPFSLN
K


In [74]:
print('Extracting DNA sequences and translating')
dnaextension = 900  # number of nucleotides to consider when extending peptides after +1 frameshift
minPeptideLength = 10
output = []
outputPlusOneAAOnly = []
outputProteinAndPlusOneAA = []
outputPlusOneAAOnlyCutoff = []
outputProteinAndPlusOneAACutoff = []
for a in annotation:
    if a[6] == '+':
        start, end = int(a[3]), int(a[4])
        PlusOneAAseq = translate_AA_from_dna_human_terminate_at_stop(dgenome[a[0]][end-2:end+dnaextension-2])  # end-2:end+1+peplength-3
    elif a[6] == '-':
        end, start = int(a[3]), int(a[4])  # the end coordinate is column three for reverse reads
        revcomp, rev = reverse_complement_to_dna(dgenome[a[0]][end-peplength+1:end+1])  # long form # [end-2-peplength+3:end+1
        PlusOneAAseq = translate_AA_from_dna_human_terminate_at_stop(revcomp)
    else:
        print('no orientation information... a problem')
    output.append('\t'.join(a + [PlusOneAAseq]))
    protID = a[-1].split(';')[-1]  # isolating the protein ID from the annotation
    outputPlusOneAAOnly.append('>%s\n%s' % (protID, PlusOneAAseq))
    outputProteinAndPlusOneAA.append('>%s\n%s' % (protID, dproteins[protID] + PlusOneAAseq))
    if len(PlusOneAAseq) > minPeptideLength:
        outputPlusOneAAOnlyCutoff.append('>%s\n%s' % (protID, PlusOneAAseq))
        outputProteinAndPlusOneAACutoff.append('>%s\n%s' % (protID, dproteins[protID] + PlusOneAAseq))
    
    
outfile = 'D:\\LinuxShare\\Humans\\Genome\\Annotations\\PlusOneFrameshift_SkipFirstStopCodonBase_AnnotAndPeptides.tsv'
with open(outfile, 'w') as OUT:
    OUT.write('\n'.join(output))
print('Output +1 frameshift peptides up to %d in length to file: %s' % (dnaextension/3, outfile))

outfile = 'D:\\LinuxShare\\Humans\\Genome\\Seqs\\PlusOneFrameshift_SkipFirstStopCodonBase_Peptides.fa'
with open(outfile, 'w') as OUT:
    OUT.write('\n'.join(outputPlusOneAAOnly))
print('Output +1 frameshift peptides up to %d in length to file: %s' % (dnaextension/3, outfile))

outfile = 'D:\\LinuxShare\\Humans\\Genome\\Seqs\\PlusOneFrameshift_SkipFirstStopCodonBase_ProteinAndPeptides.fa'
with open(outfile, 'w') as OUT:
    OUT.write('\n'.join(outputProteinAndPlusOneAA))
print('Output +1 frameshift peptides up to %d in length to file: %s' % (dnaextension/3, outfile))

outfile = 'D:\\LinuxShare\\Humans\\Genome\\Seqs\\PlusOneFrameshift_SkipFirstStopCodonBase_Peptides.Min%d.fa' % minPeptideLength
with open(outfile, 'w') as OUT:
    OUT.write('\n'.join(outputPlusOneAAOnlyCutoff))
print('Output +1 frameshift peptides up to %d in length to file: %s' % (dnaextension/3, outfile))

outfile = 'D:\\LinuxShare\\Humans\\Genome\\Seqs\\PlusOneFrameshift_SkipFirstStopCodonBase_ProteinAndPeptides.Min%d.fa' % minPeptideLength
with open(outfile, 'w') as OUT:
    OUT.write('\n'.join(outputProteinAndPlusOneAACutoff))
print('Output +1 frameshift peptides up to %d in length to file: %s' % (dnaextension/3, outfile))
    
print(datetime.datetime.now())


Extracting DNA sequences and translating
Output +1 frameshift peptides up to 300 in length to file: D:\LinuxShare\Humans\Genome\Annotations\PlusOneFrameshift_SkipFirstStopCodonBase_AnnotAndPeptides.tsv
Output +1 frameshift peptides up to 300 in length to file: D:\LinuxShare\Humans\Genome\Seqs\PlusOneFrameshift_SkipFirstStopCodonBase_Peptides.fa
Output +1 frameshift peptides up to 300 in length to file: D:\LinuxShare\Humans\Genome\Seqs\PlusOneFrameshift_SkipFirstStopCodonBase_ProteinAndPeptides.fa
Output +1 frameshift peptides up to 300 in length to file: D:\LinuxShare\Humans\Genome\Seqs\PlusOneFrameshift_SkipFirstStopCodonBase_Peptides.Min10.fa
Output +1 frameshift peptides up to 300 in length to file: D:\LinuxShare\Humans\Genome\Seqs\PlusOneFrameshift_SkipFirstStopCodonBase_ProteinAndPeptides.Min10.fa
2019-09-06 11:48:33.316973


In [ ]:
## add Plus one frameshift peptides to actaul protein sequences and output them to a fasta file
## need to keep all possible protein transcripts
## for all of these add the protein ID to the mRNA line
## example: ID=cds4;Parent=rna43;Dbxref=GeneID:105378947,Genbank:XP_011540840.1;Name=XP_011540840.1;
## extract the "XP_011540840.1"
